In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv(r'/kaggle/input/phishing-data/combined_dataset.csv')

In [ ]:
df.head()

* Domain: The URL itself.
* Ranking: Page Ranking
* isIp: Is there an IP address in the weblink
* valid: This data is fetched from google's whois API that tells us more about the current status of the URL's registration.
* activeDuration: Also from whois API. Gives the duration of the time since the registration up until now.
* urlLen: It is simply the length of the URL
* is@: If the link has a '@' character then it's value = 1
* isredirect: If the link has double dashes, there is a chance that it is a redirect. 1-> multiple dashes present together.
* haveDash: If there are any dashes in the domain name.
* domainLen: The length of just the domain name.
* noOfSubdomain: The number of subdomains preset in the URL.
* Labels: 0 -> Legitimate website , 1 -> Phishing Link/ Spam Link

# Exploratory data analysis(EDA)

In [ ]:
df.shape

In [ ]:
df.isnull().any()

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
plt.figure(figsize=(12,7))
df.boxplot()
plt.show()

In [ ]:
df['nosOfSubdomain'].value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.catplot(x="nosOfSubdomain", y="domainLen", data=df)
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
sns.barplot(x="valid", y="domainLen", data=df)
plt.show()

In [ ]:
sns.countplot(df.label)

In [ ]:
from wordcloud import WordCloud

In [ ]:
train_qs = pd.Series(df['domain'].tolist()).astype(str)

In [ ]:
cloud = WordCloud(width=1440, height=1080).generate(" ".join(train_qs.astype(str)))
plt.figure(figsize=(15, 12))
plt.imshow(cloud)
plt.axis('off')

In [ ]:
plt.figure(figsize=(15,10))
sns.lineplot(y=df.urlLen,x=df.domainLen,data=df)
plt.show()

In [ ]:
sns.countplot(df['isIp'],color="r")

# Model creation and training

In [ ]:
cor=df.corr()
cor

In [ ]:
sns.heatmap(cor)

In [ ]:
x=df.iloc[:,[1,2,3,4,5,6,7,8,9,10]]
y=df.iloc[:,[11]]

In [ ]:
x

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=4,test_size=0.2)

In [ ]:
print(x_train.shape)
print(x_test.shape)

## xgboost model

In [ ]:
import xgboost
xgb=xgboost.XGBClassifier()

In [ ]:
xgb.fit(x_train,y_train)

In [ ]:
pred=xgb.predict(x_test)
pred

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
print(accuracy_score(pred,y_test))
print(confusion_matrix(pred,y_test))
print(classification_report(pred,y_test))

## support vestor machine

In [ ]:
from sklearn import svm
s=svm.SVC()

In [ ]:
s.fit(x_train,y_train)

In [ ]:
pre=s.predict(x_test)
pre

In [ ]:
print(accuracy_score(pre,y_test))
print(confusion_matrix(pre,y_test))
print(classification_report(pre,y_test))

### so in the above taken two algorithms xgboost is best with accuracy and f1_score around ~95